In [1]:
# imports
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np 

In [2]:
# paramaters
BATCH = 128
MAX_EPOCH = 10
NOISE_SIZE = 100
#C = 3 # initial channel size for discriminator
lr = 0.0002 # Learning rate for Adam Optimiser
beta1 = 0.5 # momentum term for Adam Optimiser
beta2 = 0.9

# set random seed
torch.manual_seed(42)
np.random.seed(42)


In [ ]:
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        m.weight.data.normal_(0,0.02)
        if m.bias is not None:
            m.bias.data.zero_()
    if isinstance(m, nn.ConvTranspose2d):
        m.weight.data.normal_(0,0.02)
        if m.bias is not None:
            m.bias.data.zero_()
    if isinstance(m, nn.BatchNorm2d):
        m.weight.data.normal_(0,0.02)
        if m.bias is not None: #if whole value is not None
            m.bias.data.zero_()
        
#biases to zero, weights to normal



In [ ]:
#gen.conv_t.weight.data.normal_()


In [ ]:
# dataloader 

# load data
# test one output and visualise

In [ ]:
# model for Generator

class Gen(nn.Module): 
    def __init__(self, noise_size):
        super(Gen, self).__init__()
        # create all layers
        self.conv_t =  nn.ConvTranspose2d(noise_size, 1024, 4, stride=1, padding=0, bias=False) 
        self.conv_t1 = nn.ConvTranspose2d(1024, 512, 4, stride=2, padding=1, bias=False) 
        self.conv_t2 = nn.ConvTranspose2d(512, 256, 4, stride=2, padding=1, bias=False) 
        self.conv_t3 = nn.ConvTranspose2d(256, 128, 4, stride=2, padding=1, bias=False)
        self.conv_t4 = nn.ConvTranspose2d(128, 3, 4, stride=2, padding=1, bias=False)
        
        self.bn1 = nn.BatchNorm2d(1024)
        self.bn2 = nn.BatchNorm2d(512)
        self.bn3 = nn.BatchNorm2d(256)
        self.bn4 = nn.BatchNorm2d(128)
        self.bn5 = nn.BatchNorm2d(3)
        
        
    def forward(self, z):
        """
        Forward pass
        :param torch.Tensor z: random noise vector [B, N]
        :returns: output image [B, 3, 64, 64]
        """
        # Pass through each layer 
        # [B, N] -> [B, 1024, 4, 4]
        x = F.relu(self.bn1(self.conv_t(z)))
        # [B, 1024, 4, 4] -> [B, 512, 8, 8]
        x = F.relu(self.bn2(self.conv_t1(x)))
        # [B, 512, 8, 8] -> [B, 256, 16, 16]
        x = F.relu(self.bn3(self.conv_t2(x)))
        # [B, 256, 16, 16] -> [B, 128, 32, 32]
        x = F.relu(self.bn4(self.conv_t3(x)))
        # [B, 128, 32, 32] -> [B, 3, 64, 64]
        x = torch.tanh((self.conv_t4(x))) # images will be between -1 and 1 with tanh. NO BATCH NORM ON OUTPUT
        return x

# instanciate model
gen = Gen(NOISE_SIZE)


#print(torch.std(gen.bn1.bias)) #check via std value for conv, then for bias use min
# initialise model weights: zero-centred Normal Distr. with std dev = 0.02
gen.apply(weights_init)
#print(torch.std(gen.bn1.bias)) #as above

# test on random inputs
z = torch.randn(BATCH, NOISE_SIZE, 1, 1)

#output = model.forward(z) #another way of calling your Gen() model
outputG = gen(z)

# output = model(z)
print(outputG.shape)

# discriminator
# input will be output of Gen, so you know the size/dimensions. Need to figure out what the output of the first layer of discr will be
# then figure out how to instantiate those layers, so maybe strided conv? Keep in mind you still use Batch Norm and ReLU, they don't change
# they just affect how well your layer/training performs 

In [ ]:
# Model for Discriminator

class Discr(nn.Module): 
    def __init__(self):
        super(Discr, self).__init__()
        # Create all layers
        self.conv  = nn.Conv2d(3, 128, 4, stride=2, padding=1, bias=False) #first layer, final output of original Gen, no Transpose
        self.conv1 = nn.Conv2d(128, 256, 4, stride=2, padding=1, bias=False)
        self.conv2 = nn.Conv2d(256, 512, 4, stride=2, padding=1, bias=False)
        self.conv3 = nn.Conv2d(512, 1024, 4, stride=2, padding=1, bias=False)
        self.conv4 = nn.Conv2d(1024, 1, 4, stride=2, padding=0, bias=False)

        self.bn1 = nn.BatchNorm2d(128) #capital B because it's part of the Class
        self.bn2 = nn.BatchNorm2d(256)
        self.bn3 = nn.BatchNorm2d(512)
        self.bn4 = nn.BatchNorm2d(1024)
        
    def forward(self, image):
        """
        
        """
        # [B, 3, 64, 64] -> [B, 128, 32, 32]
        x = (self.conv(image)) # CHECK WHETHER BN IS REQUIRED -> BN REMOVED
        x = F.leaky_relu(x, negative_slope=0.2)

        x = self.bn2(self.conv1(x))
        x = F.leaky_relu(x, negative_slope=0.2)

        x = self.bn3(self.conv2(x))
        x = F.leaky_relu(x, negative_slope=0.2)

        x = self.bn4(self.conv3(x))
        x = F.leaky_relu(x, negative_slope=0.2)
        # [B, 1024, 4, 4] -> [B, 1]
        x = self.conv4(x)
        x = x.view(x.size(0), 1)
        x = torch.sigmoid(x)
        return x

# instantiate model
dis = Discr()

#print(torch.std(dis.conv.weight))
#Initialise model weights for discriminator: dis.apply
dis.apply(weights_init)
#print(torch.std(dis.conv.weight))
# test on input
image = torch.randn(BATCH, 3, 64, 64)

# output 
outputD = dis(image)
#print(outputD)

In [ ]:
# initilize ciriterion (loss function) and 2 optimizer 

criterion = nn.BCELoss()

optimiserG = torch.optim.Adam(gen.parameters(), lr, (beta1,beta2)) #Generator
optimiserD = torch.optim.Adam(dis.parameters(), lr, (beta1,beta2)) #Discriminator

#list(dis.parameters())

In [ ]:
# sample random noise for reference images
z_ref = torch.rand(BATCH, NOISE_SIZE)

with torch.no_grad():
    images = gen(z_ref)
    
# show images  

In [ ]:
# training loop
K = 1

plotting = {'loss_d':[],
           'loss_g':[]}

for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dataloader, 0): #dataload
        
        image, _ = data  # data will be assigned to the generated image and input noise z

        batch = image.size(0) #this insures it wont fail so it can take any batch size
        
        for k in range(K): #train discr twice

            # 1) train Descriminator
                optimiserD.zero_grad()
                # 1a) Train on real images

                output_real = dis(image)
                labels = torch.ones_like(output_real)
                
                loss_real = criterion(output_real, labels) #output real is output of discr
                loss_real.backward()

                # 1b) Train on generated images
                
                # generate images
                z = torch.rand(batch, NOISE_SIZE)
                image_fake = gen(z)
                
                # remove gradients from generator
                image_fake = image_fake.detach()
                
                # pass images through descriminator
                output_fake = dis(image_fake)
                labels = torch.zeros_like(output_fake)
                
                
                loss_fake = criterion(output_fake, labels)
                loss_fake.backward()

                optimiserD.step()
                
                plotting['loss_d'].append((loss_real + loss_fake).item())
            
        # 2) train Generator
        optimiserG.zero_grad()
        
        z = torch.rand(batch, NOISE_SIZE)
        
        image_gen = gen(z)
        output_gen = dis(image_gen)
        
        
        labels = torch.ones_like(output_gen)
        loss_gen = criterion(output_gen, labels)
        
        loss_gen.backward()
        optimiserG.step()
        
        plotting['loss_d'].append(loss_gen.item())
            
        #train/sample dataset/real images
        #Forward propagation function for Discr
        #Calculate loss then gradients for Discr
            #BCELoss then mini batch SGD, mini batch size 128
            
        
        #train/sample fake/Gen images
        #Forward propagation function
        #Calculate loss then gradients
            #BCELoss then mini batch SGD, mini batch size 128
        
        #Then update Generator
        #Forward prop, loss, gradients

In [ ]:
# validation loop 

In [32]:
#Discriminator step by step part

#Input size
B = 5
C = 3
H = 64
W = 64
#N = 3 #output noise dimension

# Initialise input
image = torch.randn(B, C, H, W)
#z = z.view(B, C, H, W)

# Create all layers
conv  = nn.Conv2d(3, 128, 4, stride=2, padding=1, bias=False) #first layer, final output of original Gen, no Transpose
conv1 = nn.Conv2d(128, 256, 4, stride=2, padding=1, bias=False)
conv2 = nn.Conv2d(256, 512, 4, stride=2, padding=1, bias=False)
conv3 = nn.Conv2d(512, 1024, 4, stride=2, padding=1, bias=False)
conv4 = nn.Conv2d(1024, 1, 4, stride=1, padding=0, bias=False)

bn1 = nn.BatchNorm2d(128) #capital B because it's part of the Class
bn2 = nn.BatchNorm2d(256)
bn3 = nn.BatchNorm2d(512)
bn4 = nn.BatchNorm2d(1024)

# Pass through each layer, Batch Norm 
#x = conv(image)
x = (conv(image))
x = F.leaky_relu(x, negative_slope=0.2)
print("x shape 1: ", x.shape)
x = bn2(conv1(x))
x = F.leaky_relu(x, negative_slope=0.2)
print("x shape 2: ", x.shape)
x = bn3(conv2(x))
x = F.leaky_relu(x, negative_slope=0.2)
print("x shape 3: ", x.shape)
x = bn4(conv3(x))
x = F.leaky_relu(x, negative_slope=0.2)
print("x shape 4: ", x.shape)
x = conv4(x)

# reshape x: [B, 1, 1, 1] -> [B, 1]
x = x.view(x.size(0), 1)
x = F.sigmoid(x)

# x = x.view(-1, 1)
# x = x.squeeze(-1)


print("x shape: ", x.shape)

x shape 1:  torch.Size([5, 128, 32, 32])
x shape 2:  torch.Size([5, 256, 16, 16])
x shape 3:  torch.Size([5, 512, 8, 8])
x shape 4:  torch.Size([5, 1024, 4, 4])
x shape:  torch.Size([5, 1])


In [ ]:
t = torch.randn(16,1,2,1)
print(t.squeeze(-1).shape)
print(t.squeeze(2).shape)

print(t.view(-1, 1).shape)

In [ ]:
x.squeeze(-1).size()

In [ ]:
class thing():
    
    def __init__(self, act=nn.LeakyReLU(negative_slope=0.2)):
    
        self.activation = act

In [ ]:
t = thing(act=nn.ReLU())

In [ ]:
activation = nn.LeakyReLU(negative_slope=0.2)

In [ ]:
t = torch.arange(-10,10).float()

import matplotlib.pyplot as plt
lr = activation(t)

plt.plot(t.tolist(), lr.tolist())

In [ ]:
#z = torch.randn(100) #input z
# self.z = torch.randn(4, 100) #(B,N)
# self.z = z.view(4, 100, 1, 1) #(B,N,H,W)

class Gen(nn.Module): #this is our class, the blueprint for creating an object to keep. Gen inherits from nn.Module
    def __init__(self): #this is a init method; called upon whenever you create an instance of the class
                    #using self means it connects this method to the instance of the class (Gen)
        super(Gen, self).__init__()
        # create all layers
        #these are all our attributes, being assigned to parameters/arguments
        self.conv =  nn.ConvTranspose2d(100, 1024, 4, stride=1, padding=0, bias=False) # transpose increases H,W size
        self.conv1 = nn.ConvTranspose2d(1024, 512, 4, stride=2, padding=1, bias=False) 
        self.conv2 = nn.ConvTranspose2d(512, 256, 4, stride=2, padding=1, bias=False) 
        self.conv3 = nn.ConvTranspose2d(256, 128, 4, stride=2, padding=1, bias=False)
        self.conv4 = nn.ConvTranspose2d(128, 3, 4, stride=2, padding=1, bias=False)
        
        self.bn1 = nn.BatchNorm2d(1024)
        self.bn2 = nn.BatchNorm2d(512)
        self.bn3 = nn.BatchNorm2d(256)
        self.bn4 = nn.BatchNorm2d(128)
        self.bn5 = nn.BatchNorm2d(3)
        
        # comparing with the multiclass classification tutorial with PyTorch, the hidden layers are the nn.Linear layers, which
        # are not included here
        
    def forward(self, z): #this is our method, which belongs to the object. We want to be able to reference the instance of
        # the particular object
        
        # pass through each layer 
        x = F.relu(self.bn1(self.conv(z)))
        x = F.relu(self.bn2(self.conv1(x)))
        x = F.relu(self.bn3(self.conv2(x)))
        x = F.relu(self.bn4(self.conv3(x)))
        x = F.relu(self.bn5(self.conv4(x)))
        return x

model = Gen() #this creates an instance of the class; the variable name (model) is = an instance of Gen

z= torch.randn(BATCH,NOISE_SIZE)
z = z.view(BATCH,NOISE_SIZE,1,1)

output = model(z)

print(output.shape)

In [42]:
#Input size
B = 4
C = 1024
H = 1
W = 1

N = 100 #input noise dimension Z

#Initialise input
z = torch.randn(B, N)
z = z.view(B, N, 1, 1) #reshaping the input so it has H,W
#print("z: ", z)


#Create all layers
conv =  nn.ConvTranspose2d(100, 1024, 4, stride=1, padding=0, bias=False) #input z needs to be reshaped and transposed
conv1 = nn.ConvTranspose2d(1024, 512, 4, stride=2, padding=1, bias=False) #this is a class so you instantiate it, where the variables can change
conv2 = nn.ConvTranspose2d(512, 256, 4, stride=2, padding=1, bias=False) #
conv3 = nn.ConvTranspose2d(256, 128, 4, stride=2, padding=1, bias=False)
conv4 = nn.ConvTranspose2d(128, 3, 4, stride=2, padding=1, bias=False)

bn1 = nn.BatchNorm2d(1024, momentum=None, affine=False)
bn2 = nn.BatchNorm2d(512)
bn3 = nn.BatchNorm2d(256)
bn4 = nn.BatchNorm2d(128)
bn5 = nn.BatchNorm2d(3)


#Pass through each layer, batch norm first, reLU
x = F.relu(bn1(conv(z)))
#x = F.relu(bn2(conv1(x)))
#x = F.relu(bn3(conv2(x)))
#x = F.relu(bn4(conv3(x)))
#x = F.relu(bn5(conv4(x)))
print("Final shape: ", x.shape)

# x2 = conv2(x1)
# print(x2.shape)
# x3 = conv3(x2)
# print(x3.shape)
# x4 = conv4(x3)
# print(x4.shape)

# # batch norm first then ReLU
# m = nn.ReLU()
# # anything with nn.Capital letter is a class you instantiate, so don't treat it as a function you pass variables to
#  #applied to output of x1
# inputx2 = bn1()

# outputx2 = nn.BatchNorm2d(256) #output of x2
# # inputx3 = m(x2)

# outputx3 = nn.BatchNorm2d(128) #output of x3
# # inputx4 = m(x3)

# outputx4 = nn.BatchNorm2d(3)

# #tanh
# # torch.tanh()

Final shape:  torch.Size([4, 1024, 4, 4])


In [ ]:
# create all layers
conv1 = nn.Conv2d(3,1,2)

# input size 
B = 1
C = 3
H = 64
W = 64

# initialize input
x = torch.randn(B, C, H, W)
print(t.size())

# pass through each layer 
x = conv1(x)
print(x.shape)

In [ ]:
 # With square kernels and equal stride
m = nn.Conv2d(16, 33, 3, stride=2) #this needs to be initialised first to initialise
# the weights, parameters in your network (so individual units/neurons). Remember: anything after nn. with a capital
# is a class
# non-square kernels and unequal stride and with padding
m = nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2))
# non-square kernels and unequal stride and with padding and dilation
m = nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2), dilation=(3, 1))


input = torch.randn(20, 16, 50, 100)
output = m(input)

In [ ]:
#With square kernels and equal stride
m = nn.ConvTranspose2d(16, 33, 3, stride=2)
# non-square kernels and unequal stride and with padding
m = nn.ConvTranspose2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2))
print(m)
input = torch.randn(20, 16, 50, 100)
output = m(input)

# exact output size can be also specified as an argument
input = torch.randn(1, 16, 12, 12)
downsample = nn.Conv2d(16, 16, 3, stride=2, padding=1)
upsample = nn.ConvTranspose2d(16, 16, 3, stride=2, padding=1)
h = downsample(input)
h.size()
torch.Size([1, 16, 6, 6])
output = upsample(h, output_size=input.size())
output.size()

In [ ]:
torch.randn(2,1,1)

In [ ]:
z = torch.randn(1, 1, 1, 1)

conv = nn.ConvTranspose2d(1, 1, 4, stride=1, padding=0, bias=False)

conv(z)

In [ ]:
t = torch.randn(B, 100).view(B, 100, 1, 1)

conv = nn.ConvTranspose2d(100, 1024, 4, stride=1, padding=0, bias=False)

conv(t).shape

In [ ]:
# With Learnable Parameters
m = nn.BatchNorm2d(100)
print(m)
# Without Learnable Parameters
m = nn.BatchNorm2d(100, affine=False)
input = torch.randn(20, 100, 35, 45)

output = m(input)
print(output.size())

In [ ]:
class BatchNorm:
    def __init__(self, channels):
        self.channels = channels
        self.weight = 100

    def __call__(self, x):
        return x + self.weight

In [ ]:
# python basics

# variables
var1 = 1
var2 = 'one'

print(var1, var2)

# functions

def function_name(input1, input2):
    return input1+input2

print("Function:", function_name(1,1))

# classes
print('\n','Classes:','\n')

# create a Counter class inheriting from object class (build into python)
class Counter(object):
    
    def __init__(self, start_count, count_step=1):
        
        self.count = start_count
        self.count_step = count_step
        
    def step(self):
        self.count += self.count_step
        
    def info(self):
        print(self.count)
        
    def forward(self):
        print(1)
        
    def __call__(self):
        self.forward()
        
    
days_counter = Counter(0, count_step=1)

days_counter.info()
days_counter.step()
days_counter.step()
days_counter.info()

weeks_counter = Counter(25, count_step=7)

# weeks_counter.info()
print(weeks_counter.count)
weeks_counter.step()
weeks_counter.info()

In [ ]:
weeks_counter()

In [ ]:
class SecondsCounter(Counter):
    
    def __init__(self):
        super(SecondsCounter, self).__init__(0)
        
    def info(self):
        print('Seconds:', self.count)
        
sc = SecondsCounter()

sc.info()